# NYC Apartment Search - Group 1

### Purpose of the Project:
The project uses data-driven approaches to analyze and visualize New York City apartment data, 311 complaints, and urban forestry data to help understand urban living dynamics. This analysis is intended to aid in making informed decisions about apartment rentals based on environmental and urban living conditions.

### Sections and Key Functions:
1. **Setup**
   - Initializes the environment with necessary libraries and settings.

2. **Part 1: Data Preprocessing**
   - Functions to load and clean data from various sources (ZIP codes, 311 complaints, tree census, Zillow rent data).
   - Quality checks and basic data explorations are conducted.

3. **Part 2: Storing Data**
   - Database setup functions to create tables and indices.
   - Functions to convert geometries for database insertion and to insert cleaned data into a PostgreSQL database.
   - Data retrieval functions to fetch and display samples from each database table.

4. **Part 3: Understanding the Data**
   - Functions to execute SQL queries and to extract meaningful insights from the database.
   - Various SQL queries analyze the relationship between apartment prices, complaints, and tree census data.

5. **Part 4: Visualizing the Data**
   - Multiple visualizations to represent data insights graphically, including trends over time and spatial distributions.

## Setup

In [ ]:
# Standard library imports
import os
import pathlib
import subprocess
from datetime import datetime, timedelta
from typing import Tuple

# Third-party imports
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import requests
import seaborn as sns
from sqlalchemy import create_engine
from sqlalchemy.engine.base import Engine
from shapely.geometry import Point
from geoalchemy2 import Geometry, WKTElement

In [ ]:
# Path configuration
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = DATA_DIR / "nyc_zipcodes" / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"
QUERY_DIR = pathlib.Path("queries")  # Directory for saving DB queries

# API configuration
APP_TOKEN = "J9t5fS2TcfDISWng9WsnCdvCP"
COMPLAINTS_URL = 'https://data.cityofnewyork.us/resource/erm2-nwe9.geojson'
TREES_URL = 'https://data.cityofnewyork.us/resource/uvpi-gqnh.geojson'

# Database configuration
DB_NAME = "nyc_data"
DB_USER = "williamsjs"
DB_URL = f"postgresql+psycopg2://{DB_USER}@localhost/{DB_NAME}"
engine = create_engine(DB_URL)

In [ ]:
def ensure_directory_exists(directory: pathlib.Path):
    """Ensure that a directory exists; if not, create it."""
    try:
        directory.mkdir(parents=True, exist_ok=True)
    except Exception as e:
        print(f"Error creating directory {directory}: {e}")

# Make sure the directories exist
ensure_directory_exists(DATA_DIR)
ensure_directory_exists(QUERY_DIR)

## Part 1: Data Preprocessing

The first part of the data cleaning process involves the following steps:

1. **Reading and cleaning the Zillow rental data**:
   - Loading the Zillow data from a CSV file
   - Melting the data so that each row represents a unique date-region pair
   - Filtering the data to include only New York City and the relevant date range (February 2022 to January 2024)
   - Keeping the required columns (zipcode, city, date, rent price) and renaming them
   - Converting the "zipcode" column to a string type and the "date" column to a datetime type

2. **Reading and cleaning the zipcode data**
3. **Downloading, cleaning, and filtering the 311 complaints data and tree data**
4. **Filtering all the datasets to include only the cleaned zipcode data**

5. **Performing data quality checks**:
   - Checking for null values in each dataset
   - Checking for duplicate entries in each dataset
   - Cross-referencing the zipcodes across the datasets to ensure consistency
6. **Show information and first 5 entries of each dataset**.

Overall, the purpose of this part of the code is to extract, clean, and integrate the necessary information from the original data sources, preparing the data for further analysis. It involves key steps such as data loading, data cleaning, data filtering, and data quality checks.

#### The `read_and_clean_zipcode_data()` function 
reads in a shapefile containing zipcode data, cleans and preprocesses the data, and returns a GeoDataFrame with unique zipcodes and their corresponding geometries. 

The key steps are:

1. Reading the shapefile using Geopandas, a library for working with geospatial data.
2. Selecting the relevant columns (zipcode and geometry) and renaming the 'ZIPCODE' column to 'zipcode' for better readability.
3. Converting the coordinate reference system (CRS) of the GeoDataFrame to EPSG:4326 (WGS84) for consistency.
4. Removing any duplicate zipcode entries, keeping only the first occurrence of each unique zipcode.

In [ ]:
def read_and_clean_zipcode_data() -> gpd.GeoDataFrame:
    """
    Read and clean zipcode data from a shapefile.
    
    Returns:
        GeoDataFrame: A cleaned GeoDataFrame containing unique zipcodes and their geometries.
    """
    # Read the shapefile using Geopandas
    zipcode_data = gpd.read_file(ZIPCODE_DATA_FILE)
    
    # Select relevant columns and rename them
    zipcode_cleaned = zipcode_data[['ZIPCODE', 'geometry']].rename(columns={'ZIPCODE': 'zipcode'})
    
    # Convert CRS to EPSG:4326 for consistency
    zipcode_cleaned = zipcode_cleaned.to_crs(epsg=4326)
    
    # Remove duplicate zipcodes, keeping the first occurrence
    zipcode_cleaned = zipcode_cleaned.drop_duplicates(subset=['zipcode'], keep='first')
    
    return zipcode_cleaned

#### The `download_and_clean_311_data()` function 
fetches 311 complaint data from the NYC Open Data API, cleans and preprocesses the data, and returns a GeoDataFrame containing the cleaned 311 complaints.

The key steps are:
1. Defining the API parameters to fetch 311 complaint data within a specific date range and with valid latitude/longitude coordinates.
2. Sending a request to the API and handling any errors that may occur during the download.
3. Creating a GeoDataFrame from the API response and setting the appropriate coordinate reference system (EPSG:4326).
4. Selecting and renaming the relevant columns, and dropping any rows with missing zipcodes.
5. Converting the 'created_date' column from a string to a date format.

In [ ]:
def download_and_clean_311_data() -> gpd.GeoDataFrame:
    """
    Download and clean 311 complaint data from the NYC Open Data API.
    
    Returns:
        GeoDataFrame: A cleaned GeoDataFrame containing 311 complaints with relevant fields and valid zipcodes.
    """
    # API parameters for fetching data
    complaints_params = {
        '$$app_token': APP_TOKEN,
        '$where': 'created_date >= "2022-02-01T00:00:00.000" AND created_date <= "2024-02-29T00:00:00.000" AND latitude IS NOT NULL',
        '$limit': 1000000
    }
    
    # Requesting data from the API
    complaints_response = requests.get(COMPLAINTS_URL, params=complaints_params)
    if complaints_response.status_code != 200:
        raise Exception("Failed to download data")

    # Create a GeoDataFrame from the response
    complaints_data = gpd.GeoDataFrame.from_features(complaints_response.json()['features']).set_crs(epsg=4326)
    
    # Select and rename columns, and drop rows without zipcodes
    complaints_cleaned = complaints_data[['unique_key', 'created_date', 'complaint_type', 'incident_zip', 'geometry']].rename(columns={
        'unique_key': 'unique_id', 
        'incident_zip': 'zipcode'
    }).dropna(subset=['zipcode'])
    
    # Convert 'created_date' from string to date
    complaints_cleaned['created_date'] = pd.to_datetime(complaints_cleaned['created_date']).dt.date
    
    return complaints_cleaned

#### The `download_and_clean_tree_data()` function 
fetches tree data from the NYC Open Data API, cleans and preprocesses the data, and returns a GeoDataFrame containing the cleaned tree data.

The key steps are:

1. Defining the API parameters to fetch tree data, with a limit of 10 million records.
2. Sending a request to the API and handling any errors that may occur during the download.
3. Creating a GeoDataFrame from the API response and dropping any rows where the latitude or longitude is missing.
4. Converting the latitude and longitude columns to float and creating geometry points from them.
5. Selecting and renaming the relevant columns, and dropping any rows where critical information (zipcode, health, or species) is missing.
6. Setting the coordinate reference system (EPSG:4326) for the GeoDataFrame.

In [ ]:
def download_and_clean_tree_data() -> gpd.GeoDataFrame:
    """
    Download and clean tree data from the NYC Open Data API.
    
    Returns:
        GeoDataFrame: A cleaned GeoDataFrame containing tree data with geometries created from latitude and longitude.
    """
    trees_params = {
        '$$app_token': APP_TOKEN,
        '$limit': 10000000
    }
    
    trees_response = requests.get(TREES_URL, params=trees_params)
    if trees_response.status_code != 200:
        print(f"Failed to download tree data. Status code: {trees_response.status_code}")
        return None

    # Create a GeoDataFrame from the JSON response
    trees_data = gpd.GeoDataFrame.from_features(trees_response.json())

    # Drop rows where latitude or longitude is NaN, and convert them to float
    trees_data.dropna(subset=['latitude', 'longitude'], inplace=True)
    trees_data['latitude'] = trees_data['latitude'].astype(float)
    trees_data['longitude'] = trees_data['longitude'].astype(float)

    # Create geometry points from latitude and longitude
    trees_data['geometry'] = trees_data.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)

    # Select and rename columns, drop rows where any critical information is missing
    trees_cleaned = trees_data[['tree_id', 'spc_common', 'health', 'status', 'zipcode', 'geometry']].rename(
        columns={'spc_common': 'species'}
    )
    trees_cleaned['zipcode'] = trees_cleaned['zipcode'].astype(str)
    trees_cleaned.crs = 'EPSG:4326'
    trees_cleaned.dropna(subset=['zipcode', 'health', 'species'], inplace=True)

    return trees_cleaned

#### The `read_and_clean_zillow_data()` function 
reads Zillow rental data for New York City from a CSV file, cleans and preprocesses the data, and returns a cleaned DataFrame.

The key steps are:

1. Loading the Zillow data from a CSV file into a DataFrame.
2. Melting the data so that each row represents a unique date-region pair, making it easier to work with.
3. Filtering the data to include only New York City and selecting the relevant date range (February 2022 to January 2024).
4. Keeping only the required columns (zipcode, city, data_date, rent_price) and renaming them for better readability.
5. Converting the 'zipcode' column to a string and the 'data_date' column to a datetime format.

In [ ]:
def read_and_clean_zillow_data() -> pd.DataFrame:
    """
    Read and clean Zillow rental data for New York City.
    
    Returns:
        DataFrame: A cleaned DataFrame containing Zillow rental data with selected columns and filtered dates.
    """
    # Load Zillow data from a CSV file
    zillow_data = pd.read_csv(ZILLOW_DATA_FILE)
    
    # Melt the data so that every row is a unique date-region pair
    id_vars = ['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName', 'State', 'City', 'Metro', 'CountyName']
    melted_data = pd.melt(zillow_data, id_vars=id_vars, var_name='data_date', value_name='rent_price')
    
    # Filter for New York City data and select relevant dates
    zillow_cleaned = melted_data[
        (melted_data['City'] == 'New York') &
        (melted_data['data_date'] >= '2022-02-01') &
        (melted_data['data_date'] <= '2024-01-31')
    ]
    
    # Keep only the required columns and rename them
    zillow_cleaned = zillow_cleaned[['RegionName', 'City', 'data_date', 'rent_price']].rename(
        columns={'RegionName': 'zipcode', 'City': 'city', 'data_date': 'data_date', 'rent_price': 'rent_price'}
    ).dropna()
    
    # Convert data types
    zillow_cleaned['zipcode'] = zillow_cleaned['zipcode'].astype(str)
    zillow_cleaned['data_date'] = pd.to_datetime(zillow_cleaned['data_date'])

    return zillow_cleaned

#### The `load_and_clean_all_data()` function 
is responsible for loading and cleaning multiple datasets, including zip codes, Zillow rental data, 311 complaints, and tree data. It then filters all the datasets to only include entries corresponding to the cleaned zip code data.

The key steps are:

1. Reading and cleaning the zip code data using the `read_and_clean_zipcode_data()` function.
2. Reading, cleaning, and filtering the Zillow rental data using the `read_and_clean_zillow_data()` function, ensuring that only the data for the cleaned zip codes is retained.
3. Downloading, cleaning, and filtering the 311 complaints data using the `download_and_clean_311_data()` function, again ensuring that only the data for the cleaned zip codes is kept.
4. Downloading, cleaning, and filtering the tree data using the `download_and_clean_tree_data()` function, keeping only the data for the cleaned zip codes.

In [ ]:
def load_and_clean_all_data() -> Tuple[gpd.GeoDataFrame, pd.DataFrame, gpd.GeoDataFrame, gpd.GeoDataFrame]:
    """
    Load and clean all relevant datasets including zip codes, rental data, 311 complaints, and tree data,
    filtering them to only include entries corresponding to the cleaned zip code data.
    
    Returns:
        Tuple[GeoDataFrame, DataFrame, GeoDataFrame, GeoDataFrame]: A tuple containing the cleaned data for
        zip codes, Zillow rental, 311 complaints, and tree data respectively.
    """
    # Read and clean zipcode data
    zipcode_data = read_and_clean_zipcode_data()
    
    # Read, clean and filter Zillow rental data
    zillow_data = read_and_clean_zillow_data()
    zillow_data = zillow_data[zillow_data['zipcode'].isin(zipcode_data['zipcode'])]
    
    # Download, clean and filter 311 complaints data
    complaints_data = download_and_clean_311_data()
    if complaints_data is not None:
        complaints_data = complaints_data[complaints_data['zipcode'].isin(zipcode_data['zipcode'])]
    
    # Download, clean and filter tree data
    tree_data = download_and_clean_tree_data()
    if tree_data is not None:
        tree_data = tree_data[tree_data['zipcode'].isin(zipcode_data['zipcode'])]

    return zipcode_data, zillow_data, complaints_data, tree_data

#### Load and clean all data sets

In [ ]:
# Load and clean all data sets
zipcode_data, zillow_data, complaints_data, tree_data = load_and_clean_all_data()

#### The `check_data_quality()` function 
performs a series of data quality checks on the loaded datasets, including:

1. Checking for null values in each dataset (zipcode_data, zillow_data, complaints_data, and tree_data).
2. Checking for duplicate entries in each dataset, such as duplicate zipcodes, tree IDs, and unique IDs.
3. Cross-referencing the zipcodes across the datasets to ensure that all zipcodes in the Zillow rental, 311 complaints, and tree data are present in the cleaned zipcode_data.

In [ ]:
def check_data_quality():
    """
    Perform data quality checks on the loaded datasets.
    """

    # Check for null values in each dataset
    print("Null values in zipcode_data:", zipcode_data.isnull().sum())
    print("Null values in zillow_data:", zillow_data.isnull().sum())
    if complaints_data is not None:
        print("Null values in complaints_data:", complaints_data.isnull().sum())
    if tree_data is not None:
        print("Null values in tree_data:", tree_data.isnull().sum())

    # Check for duplicate entries
    print("Duplicate zipcodes in zipcode_data:", zipcode_data['zipcode'].duplicated().sum())
    if tree_data is not None:
        print("Duplicate tree_ids in tree_data:", tree_data['tree_id'].duplicated().sum())
    if complaints_data is not None:
        print("Duplicate unique_ids in complaints_data:", complaints_data['unique_id'].duplicated().sum())
    print("Duplicate zipcodes in zillow_data:", zillow_data['zipcode'].duplicated().sum())

    # Cross-reference zipcodes across datasets
    if zillow_data is not None and zipcode_data is not None:
        print("Are all zillow_data zipcodes in zipcode_data?", all(zillow_data['zipcode'].isin(zipcode_data['zipcode'].unique())))
    if complaints_data is not None and zipcode_data is not None:
        print("Are all complaints_data zipcodes in zipcode_data?", all(complaints_data['zipcode'].isin(zipcode_data['zipcode'].unique())))
    if tree_data is not None and zipcode_data is not None:
        print("Are all tree_data zipcodes in zipcode_data?", all(tree_data['zipcode'].isin(zipcode_data['zipcode'].unique())))

# Run the data quality checks
check_data_quality()

#### Show basic info about zipcode_data

In [ ]:
zipcode_data.info()

#### Show first 5 entries about zipcode_data

In [ ]:
zipcode_data.head()

#### Show basic info about complaints_data

In [ ]:
complaints_data.info()

#### Show first 5 entries about complaints_data

In [ ]:
complaints_data.head()

#### Show basic info about tree_data

In [ ]:
tree_data.info()

#### Show first 5 entries about tree_data

In [ ]:
tree_data.head()

#### Show basic info about zillow_data

In [ ]:
zillow_data.info()

#### Show first 5 entries about zillow_data

In [ ]:
zillow_data.head()

## Part 2: Storing Data

The second part of the data storage process involves the following steps:

1. **Defining and Applying the Database Schema `(define_and_apply_schema())`**:
   - Created four tables: zipcode_data, complaints_data, tree_data, and zillow_data
   - Defined the table structure, including primary keys, foreign keys, and other constraints
   - Created spatial indexes on the geometry fields to improve query performance
   - Wrote the defined schema to an SQL file and executed it to apply the schema to the database

2. **Converting Geometries for Database Insertion `(convert_geom())`**:
   - Converted the geometry data in the GeoDataFrame to Well-Known Text (WKT) format for storage in the database

3. **Inserting Data into the Database `(insert_data())`**:
   - Used the pandas `to_sql()` method to insert the cleaned datasets into the corresponding database tables
   - Specified the data types, ensuring the geometry data is stored correctly
   - Handled any exceptions that may occur during the data insertion process

4. **Fetching Data from the Database `(fetch_data())`**:
   - Defined a generic function to use pandas' `read_sql_query()` to retrieve data from the database based on a provided SQL query
   - Provided an example usage, querying the row counts for each of the tables and returning the results

In summary, this part of the code completed the process of storing the cleaned data in a PostgreSQL database, and provided related read and write interfaces, laying the foundation for further data analysis and exploration.

#### The `create_database()` function 

performs the following key steps:

1. Creates a new PostgreSQL database named 'nyc_data' using the `createdb` command.
2. Adds the PostGIS extension to the newly created database using the `CREATE EXTENSION postgis;` SQL command.
3. If the database creation and PostGIS extension addition are successful, it prints a success message. If there is an error, it prints the error message.

In [ ]:
def create_database():
    """
    Creates a new PostgreSQL database named 'nyc_data' and adds PostGIS extension.
    """
    try:
        subprocess.run(["createdb", DB_NAME], check=True)
        subprocess.run(["psql", "--dbname", DB_NAME, "-c", "CREATE EXTENSION postgis;"], check=True)
        print("Database and PostGIS extension created successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Failed to create database or PostGIS extension: {e}")

#### The `define_and_apply_schema()` function 

performs the following key steps:

1. It defines the schema for the database, including the creation of four tables: `zipcode_data`, `complaints_data`, `tree_data`, and `zillow_data`. Each table has a specific set of columns and constraints, such as primary keys and foreign key relationships.
2. The function also creates spatial indices on the geometry columns of the tables, which will improve the performance of spatial queries.
3. The defined schema is written to a SQL file, and then executed using the `psql` command to apply the schema to the 'nyc_data' database.

In [ ]:
def define_and_apply_schema():
    """
    Defines the schema for the database and applies it using SQL file execution.
    """
    schema_sql = """
        -- Drop existing tables and indices if they exist
        DROP INDEX IF EXISTS idx_zipcode_geom CASCADE;
        DROP INDEX IF EXISTS idx_complaints_geom CASCADE;
        DROP INDEX IF EXISTS idx_tree_geom CASCADE;
        DROP TABLE IF EXISTS complaints_data CASCADE;
        DROP TABLE IF EXISTS tree_data CASCADE;
        DROP TABLE IF EXISTS zillow_data CASCADE;
        DROP TABLE IF EXISTS zipcode_data CASCADE;

        -- Create tables
        CREATE TABLE zipcode_data (
            zipcode TEXT PRIMARY KEY,
            geom GEOMETRY
        );

        CREATE TABLE complaints_data (
            unique_id BIGINT PRIMARY KEY,
            created_date DATE,
            complaint_type TEXT,
            zipcode TEXT,
            geom GEOMETRY,
            FOREIGN KEY (zipcode) REFERENCES zipcode_data(zipcode)
        );

        CREATE TABLE tree_data (
            tree_id BIGINT PRIMARY KEY,
            species TEXT,
            health TEXT,
            status TEXT,
            zipcode TEXT,
            geom GEOMETRY,
            FOREIGN KEY (zipcode) REFERENCES zipcode_data(zipcode)
        );

        CREATE TABLE zillow_data (
            zipcode TEXT,
            city TEXT,
            data_date DATE,
            rent_price NUMERIC,
            FOREIGN KEY (zipcode) REFERENCES zipcode_data(zipcode),
            PRIMARY KEY (zipcode, data_date)
        );

        -- Create spatial indices
        CREATE INDEX idx_zipcode_geom ON zipcode_data USING gist(geom);
        CREATE INDEX idx_complaints_geom ON complaints_data USING gist(geom);
        CREATE INDEX idx_tree_geom ON tree_data USING gist(geom);
    """
    
    # Write the SQL schema to a file
    with open("schema.sql", "w") as file:
        file.write(schema_sql)
    
    # Execute the SQL schema file
    try:
        subprocess.run(["psql", "-d", DB_NAME, "-f", "schema.sql"], check=True)
        print("Database schema applied successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Failed to apply database schema: {e}")
        
if __name__ == "__main__":
    create_database()
    define_and_apply_schema()

#### The `convert_geom()` function 
performs the following key steps:

1. It takes a GeoDataFrame as input, which contains a 'geometry' column with spatial data.
2. The function checks if the 'geometry' column exists in the input DataFrame.
3. If the 'geometry' column is present, it applies the `WKTElement()` function to each geometry object in the column, converting it to a Well-Known Text (WKT) representation.
4. The function then renames the 'geometry' column to 'geom' and drops the original 'geometry' column, returning a new DataFrame with the converted geometries.

In [ ]:
def convert_geom(data: gpd.GeoDataFrame) -> pd.DataFrame:
    """
    Convert geometries to WKTElement for database insertion.

    Args:
        data (GeoDataFrame): The geopandas DataFrame containing the 'geometry' column.

    Returns:
        DataFrame: A pandas DataFrame with converted geometry to WKTElement and renamed to 'geom'.
    """
    if 'geometry' in data.columns:
        data['geom'] = data['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))
        return data.drop(columns=['geometry'])
    return data

#### The `insert_data()` function 
performs the following key steps:

1. It takes a pandas DataFrame, the name of the target database table, and an optional dictionary of SQLAlchemy data types as input.
2. The function uses the `to_sql()` method of the DataFrame to insert the data into the specified table in the database.
3. The `if_exists='append'` parameter ensures that the data is appended to the table, rather than overwriting any existing data.
4. The `index=False` parameter ensures that the DataFrame index is not included in the insert.
5. The `dtype` parameter is used to specify the SQL column types for the data, which is particularly important for the 'geom' column containing the spatial data.
6. If the data insertion is successful, it prints a success message. If there is an error, it prints the error message.
7. The code calls the `insert_data()` function for each of the cleaned datasets (`zipcode_data`, `complaints_data`, `tree_data`, and `zillow_data`), using the `convert_geom()` function to convert the geometries to the appropriate format before insertion.

In [ ]:
def insert_data(df: pd.DataFrame, table_name: str, dtype: dict = None) -> None:
    """
    Inserts data into a database table.

    Args:
        df (DataFrame): The DataFrame to insert into the database.
        table_name (str): The name of the target database table.
        dtype (dict): A dictionary of SQLAlchemy types to specify SQL column types.

    Raises:
        Exception: If the data insertion fails, it raises an exception with the error message.
    """
    try:
        df.to_sql(table_name, engine, if_exists='append', index=False, dtype=dtype)
        print(f"{table_name} inserted successfully.")
    except Exception as e:
        print(f"Failed to insert {table_name}: {str(e)}")

# use the above functions to insert datasets to database
insert_data(convert_geom(zipcode_data), 'zipcode_data', {'geom': Geometry('GEOMETRY', srid=4326)})
insert_data(convert_geom(complaints_data), 'complaints_data', {'geom': Geometry('GEOMETRY', srid=4326)})
insert_data(convert_geom(tree_data), 'tree_data', {'geom': Geometry('GEOMETRY', srid=4326)})
insert_data(zillow_data, 'zillow_data')

print("Database setup and data insertion complete.")

#### The `fetch_data()` function 
performs the following key steps:

1. It takes a SQL query and a database engine connection as input.
2. The function uses the `pd.read_sql_query()` method to execute the provided SQL query and fetch the resulting data as a pandas DataFrame.
3. If there is an error during the data fetch, the function prints the error message and returns an empty DataFrame.

In [ ]:
def fetch_data(query: str, engine: Engine) -> pd.DataFrame:
    """
    Fetches data from the database based on the provided SQL query.

    Args:
        query (str): The SQL query to be executed.
        engine (Engine): The database engine connection used to execute the query.

    Returns:
        DataFrame: A DataFrame containing the results of the SQL query.
    """
    try:
        return pd.read_sql_query(query, con=engine)
    except Exception as e:
        print(f"Failed to fetch data: {str(e)}")
        return pd.DataFrame()  # Return an empty DataFrame on failure

# Example usage of the fetch_data function
query = """
    SELECT 'zipcode_data' AS table_name, COUNT(*) AS total_rows FROM zipcode_data UNION ALL
    SELECT 'complaints_data' AS table_name, COUNT(*) FROM complaints_data UNION ALL
    SELECT 'tree_data' AS table_name, COUNT(*) FROM tree_data UNION ALL
    SELECT 'zillow_data' AS table_name, COUNT(*) FROM zillow_data;
"""
row_counts = fetch_data(query, engine)

#### Print the row counts for each table

In [ ]:
row_counts

#### Fetch and print a sample from the 'zipcode_data' table

In [ ]:
zipcode_sample = fetch_data("SELECT * FROM zipcode_data LIMIT 5;", engine)
zipcode_sample

#### Fetch and print a sample from the 'complaints_data' table

In [ ]:
complaints_sample = fetch_data("SELECT * FROM complaints_data LIMIT 5;", engine)
complaints_sample

#### Fetch and print a sample from the 'tree_data' table

In [ ]:
tree_sample = fetch_data("SELECT * FROM tree_data LIMIT 5;", engine)
tree_sample

#### Fetch and print a sample from the 'zillow_data' table

In [ ]:
zillow_sample = fetch_data("SELECT * FROM zillow_data LIMIT 5;", engine)
zillow_sample

## Part 3: Understanding the Data


1. `QUERY1`: This query calculates the average rent price for each zipcode in the `zillow_data` table as of the date '2024-01-31'.

2. `QUERY2`: This query counts the number of trees for each zipcode in the `tree_data` table.

3. `QUERY3`: This query counts the number of complaints for each zipcode in the `complaints_data` table where the `created_date` is between '2024-01-01' and '2024-01-31'.

4. `QUERY4`: This query finds the correlation between rent, trees, and complaints by joining the results of the previous three queries. It focuses on the top and bottom 5 zipcodes by average rent as of '2024-01-31'.

5. `QUERY5`: This query calculates the percentage change in rent prices for each zipcode in the `zillow_data` table between the dates '2023-01-31' and '2024-01-31'.

6. `QUERY6`: This query calculates the percentage change in the number of trees for each zipcode in the `tree_data` table between the dates '2023-01-01' and '2024-01-01'.

These queries provide insights into the relationships between rent, trees, and complaints in the given data, as well as the changes in rent prices and tree counts over a one-year period from 2023 to 2024. The results can be used to analyze urban development and quality of life factors within the specified timeframe.

#### The `read_query()` function 
performs the following tasks:

1. It takes an SQL query (as a string) and a database engine connection as input parameters.
2. It attempts to execute the given SQL query using the provided database engine connection and returns the results as a pandas DataFrame.
3. If an exception occurs during the query execution, the function prints an error message and returns an empty pandas DataFrame instead of raising the exception.

In [ ]:
def read_query(query: str, engine: Engine) -> pd.DataFrame:
    """
    Executes an SQL query and returns the results as a DataFrame.

    Args:
        query (str): The SQL query to execute.
        engine (Engine): The database engine connection to use for the query.

    Returns:
        DataFrame: A DataFrame containing the query results.
    """
    try:
        return pd.read_sql(query, con=engine)
    except Exception as e:
        print(f"Failed to execute query: {str(e)}")
        return pd.DataFrame()  # Return an empty DataFrame on error

### Query 1

1. It defines an SQL query, `QUERY1`, that retrieves the number of 311 complaints per zipcode in New York City for the period between March 2023 and February 2024. The query groups the complaints by zipcode and orders the results by the complaint count in descending order.

2. It writes the `QUERY1` to a SQL file within the specified `QUERY_DIR` directory. If the directory does not exist, it creates the directory and its parent directories as needed.

3. It executes the `QUERY1` using the `read_query()` function, which takes the query and the database engine connection as input, and returns the results as a pandas DataFrame, which is stored in the `complaint_counts` variable.

4. Finally, it returns the `complaint_counts` DataFrame, which can be used for further analysis or processing.

In [ ]:
# Query 1: Number of 311 complaints per zip code
# Define the query for the number of 311 complaints per zip code
QUERY1 = """
    SELECT zipcode, COUNT(*) as complaint_count
    FROM complaints_data
    WHERE created_date >= '2023-03-01' AND created_date <= '2024-02-29'
    GROUP BY zipcode
    ORDER BY complaint_count DESC
"""

# Write the query to a SQL file within the specified directory
query_file = QUERY_DIR / "number_of_complaints_per_zipcode.sql"
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists
with open(query_file, "w") as file:
    file.write(QUERY1)

# Execute the query and fetch the results
complaint_counts = read_query(QUERY1, engine)
complaint_counts

### Query 2

1. It defines an SQL query, `QUERY2`, that retrieves the top 10 zipcodes by tree count. The query groups the tree data by zipcode, counts the number of trees for each zipcode, orders the results by the tree count in descending order, and limits the output to the top 10 rows.

2. It writes the `QUERY2` to a SQL file within the specified `QUERY_DIR` directory. If the directory does not exist, it creates the directory and its parent directories as needed.

3. It executes the `QUERY2` using the `read_query()` function, which takes the query and the database engine connection as input, and returns the results as a pandas DataFrame, which is stored in the `top_tree_counts` variable.

4. Finally, it returns the `top_tree_counts` DataFrame, which can be used for further analysis or processing.

In [ ]:
#Query 2: Top 10 zip codes by tree count
# Define the query for the top 10 zip codes by tree count
QUERY2 = """
    SELECT zipcode, COUNT(*) as tree_count
    FROM tree_data
    GROUP BY zipcode
    ORDER BY tree_count DESC
    LIMIT 10
"""

# Write the query to a SQL file within the specified directory
query_file2 = QUERY_DIR / "top_10_zipcodes_by_tree_count.sql"
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists
with open(query_file2, "w") as file:
    file.write(QUERY2)

# Execute the query and fetch the results
top_tree_counts = read_query(QUERY2, engine)
top_tree_counts

### Query 3

1. It defines an SQL query, `QUERY3`, that retrieves the average rent in the top 10 zipcodes with the most trees. The query uses a common table expression (CTE) to first identify the top 10 zipcodes by tree count, and then joins this with the Zillow rental data to calculate the average rent for each of those top 10 zipcodes.

2. It writes the `QUERY3` to a SQL file within the specified `QUERY_DIR` directory. If the directory does not exist, it creates the directory and its parent directories as needed.

3. It executes the `QUERY3` using the `read_query()` function, which takes the query and the database engine connection as input, and returns the results as a pandas DataFrame, which is stored in the `average_rent_in_greenest_areas` variable.

4. Finally, it returns the `average_rent_in_greenest_areas` DataFrame, which can be used for further analysis or processing.

In [ ]:
#Query 3: Average rent in zip codes with the most trees
# Define the query for the average rent in zip codes with the most trees
QUERY3 = """
    WITH TopTreeZipCodes AS (
        SELECT zipcode, COUNT(*) as tree_count
        FROM tree_data
        GROUP BY zipcode
        ORDER BY tree_count DESC
        LIMIT 10
    )
    SELECT t.zipcode, TO_CHAR(AVG(z.rent_price), 'FM9,999,999.00') as average_rent
    FROM TopTreeZipCodes t
    JOIN zillow_data z ON t.zipcode = z.zipcode AND z.data_date = '2024-01-31'
    GROUP BY t.zipcode, t.tree_count
    ORDER BY t.tree_count DESC
"""

# Write the query to a SQL file within the specified directory
query_file3 = QUERY_DIR / "average_rent_in_greenest_areas.sql"
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists
with open(query_file3, "w") as file:
    file.write(QUERY3)

# Execute the query and fetch the results
average_rent_in_greenest_areas = read_query(QUERY3, engine)
average_rent_in_greenest_areas

### Query 4

1. It defines an SQL query, `QUERY4`, that retrieves the correlation between rent, trees, and complaints for the top and bottom 5 zipcodes by average rent. The query uses three common table expressions (CTEs) to:
   - Calculate the average rent and rent ranking for each zipcode
   - Count the number of trees for each zipcode
   - Count the number of complaints for each zipcode in January 2024

2. It joins the results of these three CTEs to create a final result set that includes the zipcode, average rent, tree count, and complaint count for the top and bottom 5 zipcodes by average rent.

3. It writes the `QUERY4` to a SQL file within the specified `QUERY_DIR` directory. If the directory does not exist, it creates the directory and its parent directories as needed.

4. It executes the `QUERY4` using the `read_query()` function, which takes the query and the database engine connection as input, and returns the results as a pandas DataFrame, which is stored in the `correlation_results` variable.

5. Finally, it returns the `correlation_results` DataFrame, which can be used for further analysis or processing.

In [ ]:
# Query 4: Rent, trees, and complaints correlation
# Define the query for correlating rent, trees, and complaints
QUERY4 = """
    WITH RentRank AS (
        SELECT zipcode, AVG(rent_price) as average_rent,
        RANK() OVER (ORDER BY AVG(rent_price) DESC) rent_rank_desc,
        RANK() OVER (ORDER BY AVG(rent_price) ASC) rent_rank_asc
        FROM zillow_data
        WHERE data_date = '2024-01-31'
        GROUP BY zipcode
    ),
    TreeCount AS (
        SELECT zipcode, COUNT(*) as tree_count
        FROM tree_data
        GROUP BY zipcode
    ),
    ComplaintCount AS (
        SELECT zipcode, COUNT(*) as complaint_count
        FROM complaints_data
        WHERE created_date >= '2024-01-01' AND created_date < '2024-02-01'
        GROUP BY zipcode
    )
    SELECT r.zipcode,
    TO_CHAR(r.average_rent, 'FM9,999,999.00') as average_rent,
    t.tree_count,
    c.complaint_count
    FROM RentRank r
    JOIN TreeCount t ON r.zipcode = t.zipcode
    LEFT JOIN ComplaintCount c ON r.zipcode = c.zipcode
    WHERE r.rent_rank_desc <= 5 OR r.rent_rank_asc <= 5
    ORDER BY r.average_rent DESC
"""

# Write the query to a SQL file within the specified directory
query_file4 = QUERY_DIR / "rent_trees_complaints_correlation.sql"
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists
with open(query_file4, "w") as file:
    file.write(QUERY4)

# Execute the query and fetch the results
correlation_results = read_query(QUERY4, engine)
correlation_results

### Query 5

1. It defines an SQL query, `QUERY5`, that retrieves the top 10 zipcodes with the most greenery (trees) using a spatial join. The query uses a common table expression (CTE) to:
   - Join the `tree_data` and `zipcode_data` tables using a spatial containment operation (`ST_Contains`) to determine which trees belong to each zipcode.
   - Count the number of trees for each zipcode.

2. It writes the `QUERY5` to a SQL file within the specified `QUERY_DIR` directory. If the directory does not exist, it creates the directory and its parent directories as needed.

3. It executes the `QUERY5` using the `read_query()` function, which takes the query and the database engine connection as input, and returns the results as a pandas DataFrame, which is stored in the `most_greenery_results` variable.

4. Finally, it returns the `most_greenery_results` DataFrame, which can be used for further analysis or processing.

In [ ]:
# Query 5: Most greenery (take 2) using spatial join
# Define the query to find the zip codes with the most greenery using a spatial join
QUERY5 = """
    WITH TreeCount AS (
        SELECT z.zipcode, COUNT(*) as tree_count
        FROM tree_data t
        JOIN zipcode_data z ON ST_Contains(z.geom, t.geom)
        GROUP BY z.zipcode
    )
    SELECT zipcode, tree_count
    FROM TreeCount
    ORDER BY tree_count DESC
    LIMIT 10
"""

# Write the query to a SQL file within the specified directory
query_file5 = QUERY_DIR / "most_greenery_spatial.sql"
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists
with open(query_file5, "w") as file:
    file.write(QUERY5)

# Execute the query and fetch the results
most_greenery_results = read_query(QUERY5, engine)
most_greenery_results

### Query 6
1. It defines a central point using the `Point` class from the `shapely.geometry` module, with the coordinates (-73.96253174434912, 40.80737875669467).

2. It creates a buffer around the central point with a radius of 0.5 miles (approximately 804.672 meters) using the `buffer()` method of the `Point` object. This creates a circular area around the central point.

3. It constructs the SQL query `QUERY6` that retrieves the tree data (tree_id, species, health, status, and location) for all the trees within the buffered area. The `ST_DWithin()` function is used to check if the tree's geometry (`geom`) is within the buffered area.

4. It writes the `QUERY6` to a SQL file within the specified `QUERY_DIR` directory. If the directory does not exist, it creates the directory and its parent directories as needed.

5. It executes the `QUERY6` using the `read_query()` function, which takes the query and the database engine connection as input, and returns the results as a pandas DataFrame, which is stored in the `trees_within_half_mile` variable.

6. Finally, it returns the `trees_within_half_mile` DataFrame, which can be used for further analysis or processing.

In [ ]:
#Query 6: Trees within ½ mile radius
# Define the central point
central_point = Point(-73.96253174434912, 40.80737875669467)

# Buffer the central point by 0.5 miles (approximately 804.672 meters)
buffered_point = central_point.buffer(0.5 / 69)  # simple degree approximation

# Construct the query using the buffered area
QUERY6 = f"""
    SELECT tree_id, species, health, status, ST_AsText(geom) as location
    FROM tree_data
    WHERE ST_DWithin(geom, ST_GeomFromText('{buffered_point.wkt}', 4326), 804.672);
"""

# Write the query to a SQL file within the specified directory
query_file6 = QUERY_DIR / "trees_within_half_mile.sql"
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists
with open(query_file6, "w") as file:
    file.write(QUERY6)

# Execute the query and fetch the results
trees_within_half_mile = read_query(QUERY6, engine)
trees_within_half_mile